In [ ]:
def calc_weighted_mean_single(x, dx):
    """
    returns: weighted mean, error on mean, Ndof, Chi2, p_val
    """
    assert(len(x) > 1)
    assert(len(x) == len(dx))
    
    var = 1 / np.sum(1 / dx ** 2)
    mean = np.sum(x / dx ** 2) * var

    # Calculate statistics
    Ndof = len(x) - 1
    chi2 = np.sum((x - mean) ** 2 / dx ** 2)
    p_val = stats.chi2.sf(chi2, Ndof)

    return mean, np.sqrt(var), Ndof, chi2, p_val

In [ ]:
frame_range = [0,None]
for i, act in enumerate(act_list):
    vals = sfac[frame_range[0]:frame_range[1], 0, 0, i, :].reshape(-1)
    std = sfac[frame_range[0]:frame_range[1], 0, 1, i, :].reshape(-1)

  #  print(len(vals))
    vmax = np.quantile(vals, 0.95)
    vmin = np.quantile(vals, 0.05)
    if vmin > 0:
        mask = (vals > vmin) & (vals < vmax)
       # mask = std / vals < 0.3
        print("FRACTION USED ", mask.sum()/len(mask))

        m, s, Ndof, chi2, pval = calc_weighted_mean_single(vals[mask], std[mask])
        mf, sf = calc_weighted_mean_single(vals, std)[:2]
    else:
        continue
    if m > 0:
        print(act)
        print(chi2/Ndof, pval)
        print("weighted mean sort: ", np.round(mf,3), "\u00B1", np.round(sf,3))
        print("weighted mean: ", np.round(m,3), "\u00B1", np.round(s,3))
        print("mean sort: ", np.round(np.mean(vals[mask]),3), np.round(np.std(vals[mask])/np.sqrt(len(vals[mask])),3))
        print("mean: ", np.round(np.mean(vals),3), np.round(np.std(vals)/np.sqrt(len(vals)),3))
    else:
        continue

In [ ]:
kbins, sfac_av, rad, pcf_av = ad.get_sfac(use_merged=False, time_av=False)
sfac = np.load(f'data\\nematic_analysis{LL}_LL0.05\\short\\sfac.npy')
#act_list = np.load(f'data\\nematic_analysis{LL}_LL0.05\\short\\act_list_alpha_fit_sfac.npy')
act_list = np.loadtxt(f'data\\nematic_analysis{LL}_LL0.05\\short\\activity_list.txt')
sfav = np.nan * np.zeros([sfac.shape[1], sfac.shape[3], 2])
sfav_av = np.nanmean(sfac[:,:,0,:,:], axis = -1)
sfav_std = np.nanstd(sfac[:,:,0,:,:], axis = -1, ddof=1) / np.sqrt(sfac.shape[-1])



sfav_time_av, sfav_time_std = calc_weighted_mean(sfav_av, sfav_std, axis = 0)

sfav[:, :, 0] = sfav_time_av
sfav[:, :, 1] = sfav_time_std

fig,ax = sfac_plotter(list(act_list), kbins, sfav, Npoints_to_fit=8, act_idx_bounds=act_idx_bounds)
for axx in ax:
    axx.vlines(0.02, 0.3,1, color='red', linestyles='dashed')

sfac_plotter(list(act_list), kbins, sfav, Npoints_to_fit=8, act_idx_bounds=act_idx_bounds)
sw_av, sw_std = calc_weighted_mean(sfac[:,:,0,:,:], sfac[:,:,1,:,:], axis = -1)

In [ ]:


kbins, sfac_av, rad, pcf_av = ad.get_sfac(use_merged=False, time_av=False)
sfac = np.load(f'data\\nematic_analysis{LL}_LL0.05\\short\\sfac.npy')

act_idx = -2
frame = 20

fig, ax = plot_structure_factor(kbins, sfac[frame, :, 0, act_idx, 0], sfac[frame, :, 0, act_idx, 0])

ax.set_yscale('linear')
ax.set_ylim([-1, 4])

for exp in range(1,10):
    ax.errorbar(kbins, sfac[frame, :, 0, act_idx, exp], yerr = sfac[frame, :, 0, act_idx, exp], fmt = '.-', \
                        alpha = .4, markersize = 5, label = f'exp={exp}')
ax.legend(fontsize=6)

In [ ]:
    def analyze_sfac_time_av(self, Ndataset = 0, Npoints_to_fit = 5, act_idx_bounds = None, weighted = True, use_merged = False, save = True, plot = True):
        """
        returns fit_params_time_av
        """
        if use_merged:
            output_path = os.path.join(self.output_main_path, 'merged_results')
            Ndataset = np.argmin(self.priorities)

            if not os.path.isdir(output_path):
                print(f'Merged results not found. Run merge_results first.')
                return
        else:
            output_path = self.output_paths[Ndataset]

        suffix = '' if weighted else '_unweighted'

        if act_idx_bounds is None:
            act_idx_bounds = [0, len(self.act_list[Ndataset])]

        act_list = self.act_list[Ndataset] #[act_idx_bounds[0]:act_idx_bounds[1]]

        try:
            kbins = np.loadtxt(os.path.join(output_path, 'kbins.txt'))
            sfac_av = np.load(os.path.join(output_path, f'sfac_time_av{suffix}.npy'))[:, :, :]
        except:
            print('Time-averaged structure factor or pcf not found. Analyse defects first.')
            return

        def fit_func(x, alpha, beta):
                    return beta + alpha * x
        param_guess = np.array([0.1, 0.1])
        fit_string = rf'$y = \beta + \alpha |k|$'
        Nparams = len(param_guess)

        fit_params_sfac_time_av = np.zeros([len(act_list), 2 * Nparams]) * np.nan

        for i, act in enumerate(act_list):
            try:
                x = np.log(kbins[:Npoints_to_fit])
                y = np.log(sfac_av[:Npoints_to_fit, i, 0])
                yerr = sfac_av[:Npoints_to_fit, i, 1] / sfac_av[:Npoints_to_fit, i, 0]
            except:
                continue
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", category=np.VisibleDeprecationWarning)
                fit = do_chi2_fit(fit_func, x, y, yerr, param_guess, verbose = False)

            fit_params_sfac_time_av[i, :Nparams] = fit.values[:]
            fit_params_sfac_time_av[i, Nparams:] = fit.errors[:]

        if save:
            np.save(os.path.join(output_path, f'fit_params_sfac_time_av{suffix}.npy'), fit_params_sfac_time_av)
        if plot:

            ncols = 4
            nrows = int(np.ceil(len(act_list[act_idx_bounds[0]:act_idx_bounds[1]]) / ncols))
            height = nrows * 3
            fig, ax = plt.subplots(nrows = nrows, ncols = ncols, figsize=(20,height))
            ax = ax.flatten()  

            k_begin_lines_idx = Npoints_to_fit - 1
  
            s_av = sfac_av[:, :, 0]
            s_std = sfac_av[:, :, 1]
            kmin, kmax = np.nanmin(kbins), np.nanmax(kbins)
            x = kbins[:Npoints_to_fit]

            for i, act in enumerate(act_list[act_idx_bounds[0]: act_idx_bounds[1]]):
                act_idx = act_list.index(act)
                sf_min, sf_max = np.nanmin(s_av[:, act_idx]), np.nanmax(s_av[:, act_idx])           
                fit_vals = fit_func(np.log(x), *fit_params_sfac_time_av[act_idx, :Nparams])
                
                if i == 0:
                    ax[i].hlines(1, x[0], kmax, label=r'Possion', linestyles='dashed', colors='k')
                    ax[i].errorbar(kbins, s_av[:, act_idx], yerr = s_std[:, act_idx], fmt = 's', \
                                alpha = .6, color = 'blue', ecolor = 'black', markersize = 5, label = 'Binned mean')
                    ax[i].plot(x, s_av[k_begin_lines_idx, act_idx] * x**0.1 / x[-1]**0.1, '--', label=r'$k^{0.1}$',alpha=0.5,)
                    ax[i].plot(x, s_av[k_begin_lines_idx, act_idx] * x**0.2 / x[-1]**0.2, '--', label=r'$k^{0.2}$',alpha=0.5,)
                    ax[i].plot(x, s_av[k_begin_lines_idx, act_idx] * x**0.3 /x[-1]**0.3, '--', label=r'$k^{0.3}$', alpha=0.5,)
                    ax[i].plot(x, np.exp(fit_vals), 'r-', label=rf'Fit: {fit_string}')                  
                else:
                    ax[i].errorbar(kbins, s_av[:, act_idx], yerr = s_std[:, act_idx], fmt = 's', \
                                alpha = .6, color = 'blue', ecolor = 'black', markersize = 5,)
                    ax[i].hlines(1, x[0], kmax, linestyles='dashed', colors='k')
                    ax[i].plot(x, s_av[k_begin_lines_idx, act_idx] * x**0.1 / x[-1]**0.1, '--', alpha=0.5)
                    ax[i].plot(x, s_av[k_begin_lines_idx, act_idx] * x**0.2 / x[-1]**0.2, '--', alpha=0.5)
                    ax[i].plot(x, s_av[k_begin_lines_idx, act_idx] * x**0.3 /x[-1]**0.3, '--', alpha=0.5)
                    ax[i].plot(x, np.exp(fit_vals), 'r-')
                    
                ax[i].text(0.65, 0.2, rf'$\zeta$ = {act}', transform=ax[i].transAxes, fontsize=14, verticalalignment='top')
                ax[i].text(0.55, 0.4, rf'$\alpha$ = {fit_params_sfac_time_av[act_idx,0]:.2f} $\pm$ {fit_params_sfac_time_av[act_idx,Nparams]:.2f}', transform=ax[i].transAxes, fontsize=14, verticalalignment='top')  
                ax[i].set_xscale('log')
                ax[i].set_yscale('log')

                ax[i].set_xticks([kmin, 0.1, kmax], [np.round(kmin,2), 0.1, np.round(kmax,1)])
                ax[i].set_yticks([0.3, 0.4, 0.6, 1], [0.3, 0.4, 0.6, 1])

            prefix = 'Weighted' if weighted else 'Unweighted'
            fig.suptitle(fr'Structure factor different activities', y=1.03)
            fig.supxlabel(r'$|k|$')
            fig.supylabel(rf'$S(k)$ ({prefix})', x = 0)
            fig.legend(ncol=6, fontsize = 14, bbox_to_anchor=(0.8, 1.01))
            fig.tight_layout()
        return fit_params_sfac_time_av
  